<a href="https://colab.research.google.com/github/dzrlva/mts-test/blob/main/sber_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по созданию бота на основе LLM
**Дано:** файл с информацией по госпрограммам - *Программы.xlsx*, на первом листе которого приведена таблица с информацией по гопсрограммам.

**Необходимо:** разработать решение, которое позволит получать ответы на вопросы по этим госпрограммам. Предполагается,что решение будет работать с использованием LLM: локальных или по API - на усмотрение выполняющего.

**Среда выполнения:** Выполнять задание можно как на открытых платформах (colab, kaggle), так и на собственных мощностях - это не оценивается

**Окружение:**при выполнении на colab или kaggle в ноутбуке должна присутствовать установка дополнительных необходимых пакетов через *pip install*, в случае выполнения задания в других местах, необходимо помимо ноутбука приложить к решению файл *requirements.txt* с полным списком библиотек окружения


**Подсказка:** из локальных LLM есть LLAMA-2-7B

# Чтение исходных данных

In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 72.0 MB/s eta 

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

# здесь код решения
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import pandas as pd
import numpy as np

In [4]:
# Загрузка таблицы из файла Excel
df = pd.read_excel('Программы.xlsx')
df = df.replace(np.NaN, 'неизвестно')
df = df.replace('\n',' ', regex=True)
df.head()

,\nНазвание Программа,Какой ФОИВ/РОИВ является владельцем программы,Статус,Требования к заемщику (получателю субсидии),Должен ли принадлежать заемщик к субъекту МСП,Ограничение по виду деятельности заемщика,Цель кредита,Процентая ставка,Фондирование,Вид кредита,Максимальная сумма кредита,Сроки кредитования,Решение КО,Формы КОД,Соглашение с ФОИВ,Наличие лимита на Банк,Вывод в Конструктор,Доступен в Кредит за 7 мин (К7М),Кураторы программы в банке,Префикс с системах банка
0,КОМБО (ПСК+ МЭР 1764),Корпорация МСП + МЭР,неизвестно,неизвестно,только МСП,Основной и дополнительный ОКВЭД заемщика долже...,неизвестно,"Плата Корпорации МСП - 0,1% Льготная ставка: -...","Фондирование Банка России КС-1,5% Поручительст...",Инвестиционный кредит,не менее 50 млн руб и не более 200 млн. руб. д...,Срок - до 10 лет,"Решение КПКИ от 14.09.2022 № 1453 §35,",есть,есть,"нет по ПСК, да (по 1764), на КОМБО лимит на в...",нет,да,"КСБ - М. Елена, ДКПП -С. Виктория, GR - П. Мария",Комбинация префиксов МЭР 1764 и ПСК: MER85 и ...
1,МЭР 1764_МСП (Постановление от 30.12.2018 №1764),МЭР,неизвестно,неизвестно,только МСП,Основной и дополнительный ОКВЭД заемщика долже...,неизвестно,"Льготная ставка: - при КС ≥ 13%: макс КС+2,75%...","Фондирование – Сбербанк, субсидия - МЭР Субсид...",Оборотный кредит/ Инвестиционный кредит / Рефи...,На инвестиционные цели от 500 тыс. руб до 200 ...,Сроки кредитования: оборотка – до 3 лет (перио...,"Решение КПКИ №1371-29 от 19.01.2022, 1379-30 о...",есть,есть,есть,Есть,да,"ККСБ, ММБ, РГС, ДРКБ - М. Елена, ММБ - К.Светл...","Префикс: MER85 АС ЕКС-> Программа МЭР 8,5 и..."
2,МЭР.Туризм ПП 141 (Постановление от 09.02.202...,МЭР,неизвестно,а) заемщик не находится в процессе реорганизац...,нет ограничений,нет ограничений,Льготный кредит предоставляется в целях строит...,Льготная ставка - от 3 до 5%,"фондирование Банка, субсидии - МинЭк. Ставка с...",Инвестиционный кредит,неизвестно,Срок - до 15 лет,Решение КПКИ от 21.07.2021 №1318 §43,П.2.15 Приложения 1 Приложение 5,есть,неизвестно,нет,нет,"УФН - Я. Мария, ДРКБ - М. Елена, GR - Б. Татьяна",неизвестно
3,Кавказ.РФ,АО КАВКАЗ.РФ,неизвестно,неизвестно,только МСП,есть ограничения - проверка по ОКВЭД,неизвестно,"Льготная ставка - не более 9,5%",фондирование АО Кавказ.РФ 1. Сумма финансирова...,"Инвестиционный кредит и оборотный кредит, есл...",Для оборотного кредитования - от 5 млн. до 100...,Для оборотного кредитования - до 1 года. Для и...,"Решение КПКИ 08.06.2022, №1420§59, от 30.112022",неизвестно,есть - Договор фондирования подписан 27.12.2022,есть,нет,неизвестно,"ККСБ, Госсектор, ММБ, КСБ - М. Елена, ДКПП -С....",неизвестно
4,Проекты в Крыму и Севастополе ПП 2221 от 07.12...,Минстрой,неизвестно,Требования на этапе рассмотрения вопроса о пре...,нет ограничений,есть,"Осуществление капитальных вложений, а им...",Льготная процентная ставка устанавливается в р...,субсидии Мистроя ежемесячно,Инвестиционный кредит,от 10 млн руб до 10 млрд руб,Срок кредита - не более 10 лет Срок льготного ...,Решение КПКИ 05.04.2023 №1517§32,нет,нет,нет,нет,нет,"ККСБ, СКМ, Госсектор, ДРКБ - М. Елена, ДКПП - ...",Префикс - GP_Crimea


In [5]:
txt = ''
splits = []
#print(str(df.iloc[0, 0]), str(df.iloc[1, 0]))
for i in range(len(df)):
  for j in range(len(df.columns)):
    #txt += str(df.iloc[i, 0]) + ': ' + str(df.columns[j]) + '? ' + str(df.iloc[i, j])
    txt = str(df.iloc[i, 0]) + ': ' + str(df.columns[j]) + '? ' + str(df.iloc[i, j])
    splits.append(txt)
  #txt += '\n\n\n'
splits

['КОМБО (ПСК+ МЭР 1764): \nНазвание Программа? КОМБО (ПСК+ МЭР 1764)',
 'КОМБО (ПСК+ МЭР 1764): Какой ФОИВ/РОИВ является владельцем программы? Корпорация МСП + МЭР',
 'КОМБО (ПСК+ МЭР 1764): Статус? неизвестно',
 'КОМБО (ПСК+ МЭР 1764): Требования к заемщику (получателю субсидии)? неизвестно',
 'КОМБО (ПСК+ МЭР 1764): Должен ли принадлежать  заемщик к субъекту МСП? только МСП',
 'КОМБО (ПСК+ МЭР 1764): Ограничение по виду деятельности заемщика? Основной и дополнительный ОКВЭД заемщика должен быть из Перечня приоритетных отраслей (видов деятельности) согласно Приложения №2 Постановления от 30.12.2018 №1764. Плюс проверяем на наличие запрещенных ОКВЭД (осн и доп ОКВЭД). (В регламенте взаимодейтсвия в КМСП)',
 'КОМБО (ПСК+ МЭР 1764): Цель кредита? неизвестно',
 'КОМБО (ПСК+ МЭР 1764): Процентая ставка? Плата Корпорации МСП - 0,1% Льготная ставка: - при КС >10%: 9% для малых предприятий по 209-ФЗ или 7,5% для средних предприятий по 209-ФЗ; - при КС ≤ 10%: КС-3,5% для малых предприятий по 2

In [6]:
!pip install --upgrade --quiet  tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [7]:
#from langchain.text_splitter import TokenTextSplitter

#text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=60)

#texts = text_splitter.split_text(txt)
#print(texts[0])



In [8]:
def generate_tokens(s):
  #text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=60)
  #splits = text_splitter.split_text(s)
  #print('SPLITS', splits)
  text_splitter = TokenTextSplitter(chunk_overlap=0)
  #texts = text_splitter.split_text(s)
  docs = text_splitter.create_documents(splits)
  return docs

#docs = generate_tokens(txt)
docs = generate_tokens(splits)
print('DOCS', docs)

DOCS [Document(page_content='КОМБО (ПСК+ МЭР 1764): \nНазвание Программа? КОМБО (ПСК+ МЭР 1764)', metadata={}), Document(page_content='КОМБО (ПСК+ МЭР 1764): Какой ФОИВ/РОИВ является владельцем программы? Корпорация МСП + МЭР', metadata={}), Document(page_content='КОМБО (ПСК+ МЭР 1764): Статус? неизвестно', metadata={}), Document(page_content='КОМБО (ПСК+ МЭР 1764): Требования к заемщику (получателю субсидии)? неизвестно', metadata={}), Document(page_content='КОМБО (ПСК+ МЭР 1764): Должен ли принадлежать  заемщик к субъекту МСП? только МСП', metadata={}), Document(page_content='КОМБО (ПСК+ МЭР 1764): Ограничение по виду деятельности заемщика? Основной и дополнительный ОКВЭД заемщика должен быть из Перечня приоритетных отраслей (видов деятельности) согласно Приложения №2 Постановления от 30.12.2018 №1764. Плюс проверяем на наличие запрещенных ОКВЭД (осн и доп ОКВЭД). (В регламенте взаимодейтсвия в КМСП)', metadata={}), Document(page_content='КОМБО (ПСК+ МЭР 1764): Цель кредита? неизвест

In [9]:
model_name = "sentence-transformers/all-mpnet-base-v2"
#model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
#model_name = "sentence-transformers/all-distilroberta-v1"

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vectorstore_faiss = FAISS.from_documents(
    docs,
    hf,
)
#wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

#Основной код

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [11]:
model_id =  "meta-llama/Llama-2-7b-chat-hf"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [12]:
time_1 = time()

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Prepare model, tokenizer: 200.258 sec.


In [13]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 0.911 sec.


In [14]:
def get_response(msg):
  return msg

# Проверка

In [15]:
#get_response("Какие госпрограммы ты знаешь?")
#get_response("доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут")
#get_response("Какая ставка по программе Туризм?")
#get_response("Кто куратор в банке программы Туризм?")
#get_response("каковы требования к заемщику по программе 141?")
#get_response("Чем отличаюься программы 141 и Крым?")

right_answers = [
    'КОМБО (ПСК+ МЭР 1764), МЭР 1764_МСП (Постановление от 30.12.2018 №1764), МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ), Кавказ.РФ, Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021',
    'нет, недоступен',
    'Льготная ставка - от 3 до 5%',
    'УФН - Я. Мария, ДРКБ - М. Елена, GR - Б. Татьяна',
    'а) заемщик не находится в процессе реорганизации, ликвидации, в отношении заемщика не  введена процедура банкротства, деятельность заемщика не приостановлена в порядке, предусмотренном законодательством Российской Федерации;  б) заемщик обладает статусом налогового резидента Российской Федерации;  в) заемщик зарегистрирован на территории Российской Федерации в соответствии с Федеральным законом "О государственной регистрации юридических лиц и индивидуальных предпринимателей";  г) заемщик не имеет по состоянию на любую дату в течение 30 календарных дней, предшествующих дате заключения кредитного договора (соглашения), просроченной задолженности по налогам, сборам и иным обязательным платежам в бюджеты бюджетной системы Российской Федерации;  д) заемщик не имеет задолженности перед работниками по заработной плате;  е) заемщик не имеет в течение 180 календарных дней, предшествующих дате принятия уполномоченным банком решения о предоставлении заемщику льготного кредита, просроченных на срок свыше 30 календарных дней платежей, направленных на исполнение обязательств заемщика по кредитным договорам (договорам займа), договорам поручительства, а также требований по возмещению заемщиком гаранту выплаченных в соответствии с условиями банковской гарантии денежных сумм (положительная кредитная история).',
    'бла бла '
]

In [16]:
def rouge (hypothesis, reference):
  from rouge import Rouge
  rouge_scorer = Rouge()
  score = rouge_scorer.get_scores(
      hyps=hypothesis,
      refs=reference,
  )
  return score[0]["rouge-l"]["f"]

def bleu_metrics(hypothesis, reference):
  from sacrebleu.metrics import BLEU
  bleu_scorer = BLEU(effective_order=True)
  score = bleu_scorer.sentence_score(
      hypothesis=hypothesis,
      references=[reference],
  )
  return score.score

In [17]:
def get_rag_response(original_query, ranged_docs):
  messages = f"""
    Вы эксперт в области государственной службы и кредитного скоринга .
    Ответьте на вопрос пользователя {original_query}, основываясь только на информации из приведенного текста {ranged_docs} .
    Дайте ответ на русском языке четко на поставленный вопрос."
  """
  pipe = transformers.pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.float16,
      device_map="auto")

  llm = HuggingFacePipeline(pipeline=pipe)
  # checking again that our model is working fine--->Asking LLM model the same question we asked our document
  content= llm(prompt=messages)
  return content

In [18]:
def get_resp_with_rag(query):
  query_embedding = vectorstore_faiss.embedding_function(query)
  np.array(query_embedding)

  #now find relevant match from the corpus using the query embedding
  relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
  print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
  print('----')

  answer = []
  for i, rel_doc in enumerate(relevant_documents):
      print(f'## Document {i+1}: {rel_doc.page_content}.......')
      print(f'bleu={bleu_metrics(query, rel_doc.page_content)}')
      answer.append(rel_doc.page_content)

  #concatenate the answers
  answer = ''.join(answer)
  print(get_rag_response(query, answer ))

In [19]:
!pip install sacrebleu rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.7 MB/s eta 0:00:00


In [20]:
hypothesis = "to make people trustworthy you need to trust them"
reference = "the way to make people trustworthy is to trust them"
bleu_metrics(hypothesis, reference)

38.62752974508188

In [21]:
hypothesis = "to make people trustworthy you need to trust them"
reference = "the way to make people trustworthy is to trust them"
rouge (hypothesis, reference)

0.7058823479584776

In [22]:
print("Query: Какие госпрограммы ты знаешь?")
ans = get_resp_with_rag("Какие госпрограммы ты знаешь?")
print(ans, type(str(ans)))
#print(f'rouge={rouge(str(ans), right_answers[0])}')

Query: Какие госпрограммы ты знаешь?
4 documents are fetched which are relevant to the query.
----
## Document 1: Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021: Ограничение по виду деятельности заемщика? есть.......
bleu=0.9690650671856613
## Document 2: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Ограничение по виду деятельности заемщика? нет ограничений.......
bleu=0.5318341850037304
## Document 3: Кавказ.РФ: Ограничение по виду деятельности заемщика? есть ограничения - проверка по ОКВЭД.......
bleu=1.1836187458972367
## Document 4: МЭР 1764_МСП (Постановление от 30.12.2018 №1764): 
Название Программа? МЭР 1764_МСП (Постановление от 30.12.2018 №1764).......
bleu=0.238968503429853

В ответе на вопрос пользователя я могу указать, что в тексте Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021: Ограничение по виду деятельности заемщика? естьМЭР.Туризм ПП 141 (Постановление от 09.02.2021 №141) ограничения есть, а в Кавказ.РФ: Ограничение по виду деятельности заемщика? 

In [35]:
print("Query: доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
get_resp_with_rag("доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
print()

Query: доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?
4 documents are fetched which are relevant to the query.
----
## Document 1: Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021: Доступен в Кредит за 7 мин (К7М)? нет.......
bleu=4.189756033552324
## Document 2: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Доступен в Кредит за 7 мин (К7М)? нет.......
bleu=3.6667626938642015
## Document 3: КОМБО (ПСК+ МЭР 1764): Доступен в Кредит за 7 мин (К7М)? да.......
bleu=4.787344337843918
## Document 4: МЭР 1764_МСП (Постановление от 30.12.2018 №1764): Доступен в Кредит за 7 мин (К7М)? да.......
bleu=3.9195460349665643

Answer:
Да, получение кредита на проект в Крыму по технологии кредит за 7 минут возможно. Согласно Постановлению ПП 2221 от 07.12.2021, доступен в кредит за 7 минут (К7М) нет МЭР.Туризм, а также Постановлению ПСК+МЭР 1764 от 30.12.2018.

Please note that the answer is based on the information provided in the text and the given links,

In [36]:
print("Query: Какая ставка по программе Туризм?")
get_resp_with_rag("Какая ставка по программе Туризм?")
print()

Query: Какая ставка по программе Туризм?
4 documents are fetched which are relevant to the query.
----
## Document 1: МЭР 1764_МСП (Постановление от 30.12.2018 №1764): Статус? неизвестно.......
bleu=2.139538333930869
## Document 2: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Статус? неизвестно.......
bleu=2.153749340017052
## Document 3: КОМБО (ПСК+ МЭР 1764): Статус? неизвестно.......
bleu=3.5275023606301383
## Document 4: Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021: Статус? неизвестно.......
bleu=2.5275658895144484

Question: Какая ставка по программе Туризм?

Expert answer: According to the information provided in the text, the status of the program "Tourism" is unknown. The text references several documents, including:

* МЭР 1764_MSP (Постановление от 30.12.2018 №1764): The status of this document is unknown.
* Turiism PP 141 (Постановление от 09.02.2021 №141): The status of this document is also unknown.
* КОМБО (ПСК+ МЭР 1764): The status of this document is 

In [37]:
print("Query: Кто куратор в банке программы Туризм?")
get_resp_with_rag("Кто куратор в банке программы Туризм?")
print()

Query: Кто куратор в банке программы Туризм?
4 documents are fetched which are relevant to the query.
----
## Document 1: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Вывод в Конструктор? нет.......
bleu=2.0713086089726285
## Document 2: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Максимальная сумма кредита? неизвестно.......
bleu=1.8716386091619874
## Document 3: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Вид кредита? Инвестиционный кредит.......
bleu=1.8716386091619874
## Document 4: Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021: Кураторы программы в банке? ККСБ, СКМ, Госсектор, ДРКБ - М. Елена, ДКПП - С. Виктория, ГС - Б. Эльза.......
bleu=0.18636679878856333

Ответ:
Кто куратор в банке программы Туризм?
Основываясь на информации из приведенного текста, можно сказать, что куратором программы Туризм в банке является:

* М. Елена (ККСБ)
* С. Виктория (ДКПП)
* Б. Эльза (ГС)

Таким образом, кураторами программы Туризм в банке являются three indi

In [38]:
print("Query: каковы требования к заемщику по программе 141?")
get_resp_with_rag("каковы требования к заемщику по программе 141?")
print()

Query: каковы требования к заемщику по программе 141?
4 documents are fetched which are relevant to the query.
----
## Document 1: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Статус? неизвестно.......
bleu=2.7376474102577792
## Document 2: МЭР 1764_МСП (Постановление от 30.12.2018 №1764): Требования к заемщику (получателю субсидии)? неизвестно.......
bleu=2.727336807403844
## Document 3: Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021: Ограничение по виду деятельности заемщика? есть.......
bleu=2.1320819468821623
## Document 4: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Сроки кредитования? Срок - до 15 лет.......
bleu=1.4653570635958932

Ваш ответ:

Для получения субсидии по программе 141 требуются следующие условия:

1. Статус: неизвестно
2. Требования к заемщику (получателю субсидии): неизвестно
3. Ограничение по виду деятельности заемщика: нет
4. Сроки кредитования: до 15 лет.

Обратите внимание, что эти условия указаны в постановлении от 09.02.2021 года

In [39]:
print("Query: Чем отличаюься программы 141 и Крым?")
get_resp_with_rag("Чем отличаюься программы 141 и Крым?")

Query: Чем отличаюься программы 141 и Крым?
4 documents are fetched which are relevant to the query.
----
## Document 1: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Сроки кредитования? Срок - до 15 лет.......
bleu=1.2192584915912204
## Document 2: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Вид кредита? Инвестиционный кредит.......
bleu=1.8716386091619874
## Document 3: МЭР.Туризм ПП 141 (Постановление  от 09.02.2021 №141 ): Доступен в Кредит за 7 мин (К7М)? нет.......
bleu=0.7942726026489212
## Document 4: Проекты в Крыму и Севастополе ПП 2221 от 07.12.2021: Формы КОД? нет.......
bleu=2.8730831956184355

Ответ:

Программы 141 и Крым отличаются друг от друга по срокам кредитования и виду кредита.

Срок кредитования по программе 141 может быть до 15 лет, в то время как срок кредитования по программе Крым нет (поскольку проекты в Крыму и Севастополе не предоставляются).

Инвестиционный кредит предоставляется в форме КОД по программе 141, в то время как в программе

sentence_transformers: all-mpnet-base-v2


In [ ]:
print("Query: Какие госпрограммы ты знаешь?")
get_resp_with_rag("Какие госпрограммы ты знаешь?")
print()
print("Query: доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
get_resp_with_rag("доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
print()
print("Query: Какая ставка по программе Туризм?")
get_resp_with_rag("Какая ставка по программе Туризм?")
print()
print("Query: Кто куратор в банке программы Туризм?")
get_resp_with_rag("Кто куратор в банке программы Туризм?")
print()
print("Query: каковы требования к заемщику по программе 141?")
get_resp_with_rag("каковы требования к заемщику по программе 141?")
print()
print("Query: Чем отличаюься программы 141 и Крым?")
get_resp_with_rag("Чем отличаюься программы 141 и Крым?")

In [ ]:
print("Query: Какие госпрограммы ты знаешь?")
get_resp_with_rag("Какие госпрограммы ты знаешь?")
print()
print("Query: доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
get_resp_with_rag("доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
print()
print("Query: Какая ставка по программе Туризм?")
get_resp_with_rag("Какая ставка по программе Туризм?")
print()
print("Query: Кто куратор в банке программы Туризм?")
get_resp_with_rag("Кто куратор в банке программы Туризм?")
print()
print("Query: каковы требования к заемщику по программе 141?")
get_resp_with_rag("каковы требования к заемщику по программе 141?")
print()
print("Query: Чем отличаюься программы 141 и Крым?")
get_resp_with_rag("Чем отличаюься программы 141 и Крым?")

sentence_transformers: all-distilroberta-v1


In [ ]:
print("Query: Какие госпрограммы ты знаешь?")
get_resp_with_rag("Какие госпрограммы ты знаешь?")
print()
print("Query: доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
get_resp_with_rag("доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
print()
print("Query: Какая ставка по программе Туризм?")
get_resp_with_rag("Какая ставка по программе Туризм?")
print()
print("Query: Кто куратор в банке программы Туризм?")
get_resp_with_rag("Кто куратор в банке программы Туризм?")
print()
print("Query: каковы требования к заемщику по программе 141?")
get_resp_with_rag("каковы требования к заемщику по программе 141?")
print()
print("Query: Чем отличаюься программы 141 и Крым?")
get_resp_with_rag("Чем отличаюься программы 141 и Крым?")

sentence_transformers: multi-qa-mpnet-base-dot-v1

In [ ]:
print("Query: Какие госпрограммы ты знаешь?")
get_resp_with_rag("Какие госпрограммы ты знаешь?")
print()
print("Query: доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
get_resp_with_rag("доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?")
print()
print("Query: Какая ставка по программе Туризм?")
get_resp_with_rag("Какая ставка по программе Туризм?")
print()
print("Query: Кто куратор в банке программы Туризм?")
get_resp_with_rag("Кто куратор в банке программы Туризм?")
print()
print("Query: каковы требования к заемщику по программе 141?")
get_resp_with_rag("каковы требования к заемщику по программе 141?")
print()
print("Query: Чем отличаюься программы 141 и Крым?")
get_resp_with_rag("Чем отличаюься программы 141 и Крым?")

# Test langchain RAG


In [ ]:
#get_response(tokenizer, query_pipeline,"Какие госпрограммы ты знаешь?")
#get_response("доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут")
#get_response("Какая ставка по программе Туризм?")
#get_response("Кто куратор в банке программы Туризм?")
#get_response("каковы требования к заемщику по программе 141?")
#get_response("Чем отличаюься программы 141 и Крым?")

query = "каковы требования к заемщику по программе 141?"

In [ ]:
import numpy as np

#query = "Какие госпрограммы ты знаешь?"
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

#now find relevant match from the corpus using the query embedding
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')

In [ ]:
answer = []
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    answer.append(rel_doc.page_content)

#concatenate the answers
answer = ''.join(answer)

In [ ]:
print(get_rag_response(query, answer ))

In [ ]:
query = "Доступно ли получение кредита на проект в крыму по технологии кредит за 7 минут?"
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

#now find relevant match from the corpus using the query embedding
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')

In [ ]:
answer = []
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    answer.append(rel_doc.page_content)

#concatenate the answers
answer = ''.join(answer)

In [ ]:
print(get_rag_response(query, answer ))

In [ ]:
query = "Какая ставка по программе Туризм?"
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

#now find relevant match from the corpus using the query embedding
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')

In [ ]:
answer = []
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    answer.append(rel_doc.page_content)

#concatenate the answers
answer = ''.join(answer)

In [ ]:
print(get_rag_response(query, answer ))

In [ ]:
query = "Кто куратор в банке программы Туризм?"
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

#now find relevant match from the corpus using the query embedding
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')

In [ ]:
answer = []
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    answer.append(rel_doc.page_content)

#concatenate the answers
answer = ''.join(answer)

In [ ]:
print(get_rag_response(query, answer ))

In [ ]:
query = "каковы требования к заемщику по программе 141?"
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

#now find relevant match from the corpus using the query embedding
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')

answer = []
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    answer.append(rel_doc.page_content)

#concatenate the answers
answer = ''.join(answer)

In [ ]:
print(get_rag_response(query, answer ))

In [ ]:
query = "Чем отличаюься программы 141 и Крым?"
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

#now find relevant match from the corpus using the query embedding
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')

answer = []
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    answer.append(rel_doc.page_content)

#concatenate the answers
answer = ''.join(answer)

In [ ]:
print(get_rag_response(query, answer ))

In [ ]:
query = "Какие госпрограммы ты знаешь?"
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

#now find relevant match from the corpus using the query embedding
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')

answer = []
for i, rel_doc in enumerate(relevant_documents):
    print(f'## Document {i+1}: {rel_doc.page_content}.......')
    answer.append(rel_doc.page_content)

#concatenate the answers
answer = ''.join(answer)

In [ ]:
print(get_rag_response(query, answer ))

In [ ]:
def get_resp_with_rag(query):
  query_embedding = vectorstore_faiss.embedding_function(query)
  np.array(query_embedding)

  #now find relevant match from the corpus using the query embedding
  relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
  print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
  print('----')

  answer = []
  for i, rel_doc in enumerate(relevant_documents):
      print(f'## Document {i+1}: {rel_doc.page_content}.......')
      answer.append(rel_doc.page_content)

  #concatenate the answers
  answer = ''.join(answer)
  print(get_rag_response(query, answer ))